# **Step 1: Imports & Setup**

In [ ]:
import os, random
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16, MobileNetV2, EfficientNetB0, EfficientNetB3, ResNet50V2

from sklearn.utils import class_weight
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# **Step 2: Dataset Paths**

In [ ]:
data_dir = '/kaggle/input/pothole-detection-dataset'
img_height, img_width = 224, 224
batch_size = 16
num_classes = 2
seed = 42

# reproducibility
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

# **Step 3: Count Images & Visualization**

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

# Correct paths
normal_path = os.path.join(data_dir, "normal")
potholes_path = os.path.join(data_dir, "potholes")

# Count images
normal_imgs = glob(os.path.join(normal_path, '*.jpg'))
pothole_imgs = glob(os.path.join(potholes_path, '*.jpg'))

print("Normal images:", len(normal_imgs))
print("Pothole images:", len(pothole_imgs))

# Pie chart
labels = ['Normal', 'Potholes']
sizes = [len(normal_imgs), len(pothole_imgs)]
if sum(sizes) > 0:   # avoid NaN error
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=['skyblue','salmon'])
    plt.title("Class Distribution")
    plt.show()

    # Bar plot
    sns.barplot(x=labels, y=sizes, palette='Set2')
    plt.title("Class Distribution (Bar Plot)")
    plt.show()
else:
    print("No images found. Double-check dataset format.")


# **Step 4: Data Generators**

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    brightness_range=[0.8,1.2],
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=seed
)

val_generator = val_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,   # important for matching preds <-> y_true
    seed=seed
)

# compute class weights (helps if imbalance)
classes_array = train_generator.classes
cw = class_weight.compute_class_weight('balanced', classes=np.unique(classes_array), y=classes_array)
class_weights = dict(enumerate(cw))
print("Class weights:", class_weights)

# **Step 5: Baseline CNN**

In [ ]:
def make_baseline(input_shape=(img_height,img_width,3), num_classes=num_classes):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(128, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Flatten(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

baseline_model = make_baseline()
baseline_model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

callbacks_common = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
]

ckpt_baseline = ModelCheckpoint('baseline_best.h5', monitor='val_accuracy', save_best_only=True, mode='max')
history_baseline = baseline_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,
    callbacks=[ckpt_baseline] + callbacks_common,
    class_weight=class_weights
)
baseline_model.save("baseline_cnn_model.h5")

# **Step 6: Transfer Learning**

In [ ]:
def build_transfer_model(base_model, num_classes=num_classes):
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    preds = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=preds)
    return model, base_model

def train_model(model, base_model, name, initial_epochs=12, fine_tune_epochs=8, unfreeze_at=50):
    # Phase 1: train top
    model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    ckpt_1 = ModelCheckpoint(f'{name}_best.h5', monitor='val_accuracy', save_best_only=True, mode='max')
    h1 = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=initial_epochs,
        callbacks=[ckpt_1] + callbacks_common,
        class_weight=class_weights
    )

    # Phase 2: unfreeze some top layers of base_model and fine-tune
    base_model.trainable = True
    # freeze all but top `unfreeze_at` layers
    if unfreeze_at > 0:
        for layer in base_model.layers[:-unfreeze_at]:
            layer.trainable = False

    model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    ckpt_ft = ModelCheckpoint(f'{name}_finetuned_best.h5', monitor='val_accuracy', save_best_only=True, mode='max')
    h2 = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=fine_tune_epochs,
        callbacks=[ckpt_ft] + callbacks_common,
        class_weight=class_weights
    )
    return h1, h2

# **Step 7: Train Transfer Models**

In [ ]:
# VGG16
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(img_height,img_width,3))
vgg_model, vgg_base = build_transfer_model(vgg)
history_vgg, history_vgg_ft = train_model(vgg_model, vgg_base, "vgg16_transfer_model")

# MobileNetV2
mob = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height,img_width,3))
mob_model, mob_base = build_transfer_model(mob)
history_mob, history_mob_ft = train_model(mob_model, mob_base, "mobilenetv2_transfer_model")

# EfficientNetB0
eff0 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height,img_width,3))
eff0_model, eff0_base = build_transfer_model(eff0)
history_eff0, history_eff0_ft = train_model(eff0_model, eff0_base, "efficientnetb0_transfer_model")

# # EfficientNetB3
# eff3 = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(img_height,img_width,3))
# eff3_model, eff3_base = build_transfer_model(eff3)
# history_eff3, history_eff3_ft = train_model(eff3_model, eff3_base, "efficientnetb3_transfer_model")

# ResNet50V2
resnet_base = ResNet50V2(weights='imagenet', include_top=False, input_shape=(img_height,img_width,3))
resnet_model, resnet_base = build_transfer_model(resnet_base)
history_resnet, history_resnet_ft = train_model(resnet_model, resnet_base, "resnet50v2_transfer_model", initial_epochs=12, fine_tune_epochs=8, unfreeze_at=50)

# # After training you can load the best finetuned model:
# best_model_path = 'resnet50v2_transfer_model_finetuned_best.h5'  # saved by checkpoint naming above
# if os.path.exists(best_model_path):
#     best_model = load_model(best_model_path)
# else:
#     # fallback to last saved
#     best_model = resnet_model

In [ ]:
# Dictionary of model names and checkpoint paths
model_files = {
    'Baseline CNN': ('baseline_best.h5', baseline_model),
    'VGG16 FT': ('vgg16_transfer_model_finetuned_best.h5', vgg_model),
    'MobileNetV2 FT': ('mobilenetv2_transfer_model_finetuned_best.h5', mob_model),
    'EfficientNetB0 FT': ('efficientnetb0_transfer_model_finetuned_best.h5', eff0_model),
    'ResNet50V2 FT': ('resnet50v2_transfer_model_finetuned_best.h5', resnet_model)
}

results = {}

for name, (path, model) in model_files.items():
    eval_model = load_model(path) if os.path.exists(path) else model
    loss, acc = eval_model.evaluate(val_generator, verbose=0)
    results[name] = {'loss': loss, 'accuracy': acc}
    print(f"{name} -> Loss: {loss:.4f}, Accuracy: {acc:.4f}")


In [ ]:
best_model_name = max(results, key=lambda k: results[k]['accuracy'])
best_model_path = model_files[best_model_name][0]
best_model = load_model(best_model_path) if os.path.exists(best_model_path) else model_files[best_model_name][1]

print(f"\nBest model selected: {best_model_name} with accuracy {results[best_model_name]['accuracy']:.4f}")


# **Step 8: Plot Training Results**

In [ ]:
def plot_history(history, title):
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='Train')
    plt.plot(history.history['val_accuracy'], label='Validation')
    plt.legend(); plt.title(title + " Accuracy")

    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.legend(); plt.title(title + " Loss")
    plt.show()

plot_history(history_baseline, "Baseline CNN")
plot_history(history_vgg_ft, "VGG16")
plot_history(history_mob_ft, "MobileNetV2")
plot_history(history_eff0_ft, "EfficientNetB0")
# plot_history(history_eff3_ft, "EfficientNetB3")
plot_history(history_resnet_ft, "ResNet50V2")

# **Step 9: Summary of all models**

In [ ]:
import pandas as pd

# Convert results dict to DataFrame
df_results = pd.DataFrame(results).T  # transpose so models are rows

print(df_results)


# **Step 10: Plot summary bars for accuracy and loss**

In [ ]:
import matplotlib.pyplot as plt

# Extract data for plotting
model_names = list(results.keys())
accuracies = [results[m]['accuracy'] for m in model_names]
losses = [results[m]['loss'] for m in model_names]

# Plot Accuracy
plt.figure(figsize=(10,5))
plt.bar(model_names, accuracies, color='skyblue')
plt.ylabel('Accuracy')
plt.ylim(0, 1)  # assuming accuracy between 0 and 1
plt.title('Model Accuracy Comparison')
plt.xticks(rotation=45)
plt.show()

# Plot Loss
plt.figure(figsize=(10,5))
plt.bar(model_names, losses, color='salmon')
plt.ylabel('Loss')
plt.title('Model Loss Comparison')
plt.xticks(rotation=45)
plt.show()


# **Step 11 : Compare All Models with Full Metrics**

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.models import load_model

def evaluate_model(model, val_generator, model_name, plot_roc=False):
    """
    Evaluate a model and plot results, similar to previous bar chart style.
    Stores loss and accuracy for summary comparison.
    """
    # Reset generator
    val_generator.reset()
    steps = int(np.ceil(val_generator.samples / val_generator.batch_size))
    
    # Evaluate loss & accuracy
    loss, acc = model.evaluate(val_generator, steps=steps, verbose=0)
    
    # Predictions
    preds = model.predict(val_generator, steps=steps, verbose=0)
    y_pred = np.argmax(preds, axis=1)
    y_true = val_generator.classes
    class_labels = list(val_generator.class_indices.keys())
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.title(f"Confusion Matrix - {model_name}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()
    
    # Classification Report
    print(f"\nClassification Report - {model_name}")
    print(classification_report(y_true, y_pred, target_names=class_labels, digits=4))
    
    # ROC Curve (binary only)
    if plot_roc and len(class_labels) == 2:
        fpr, tpr, _ = roc_curve(y_true, preds[:, 1])
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(6,5))
        plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.4f})', color='blue')
        plt.plot([0,1],[0,1],'k--')
        plt.xlim([0,1])
        plt.ylim([0,1.05])
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"ROC Curve - {model_name}")
        plt.legend(loc="lower right")
        plt.grid(True)
        plt.show()
    
    return {'loss': loss, 'accuracy': acc}

# Now evaluate all models and store in results dict (like before)
results = {}

model_files = {
    'Baseline CNN': ('baseline_best.h5', baseline_model),
    'VGG16 FT': ('vgg16_transfer_model_finetuned_best.h5', vgg_model),
    'MobileNetV2 FT': ('mobilenetv2_transfer_model_finetuned_best.h5', mob_model),
    'EfficientNetB0 FT': ('efficientnetb0_transfer_model_finetuned_best.h5', eff0_model),
    # 'EfficientNetB3 FT': ('efficientnetb3_transfer_model_finetuned_best.h5', eff3_model),
    'ResNet50V2 FT': ('resnet50v2_transfer_model_finetuned_best.h5', resnet_model)
}

for name, (path, model) in model_files.items():
    eval_model = load_model(path) if os.path.exists(path) else model
    results[name] = evaluate_model(eval_model, val_generator, name, plot_roc=True)

In [ ]:
evaluate_model(best_model, val_generator, best_model_name, plot_roc=True)

# **Step 12: Grad-CAM Visualization****

In [ ]:
def find_last_conv_layer_name(model):
    # search top-level first, then nested submodels
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            return layer.name
        if hasattr(layer, 'layers'):
            for sub in reversed(layer.layers):
                if isinstance(sub, tf.keras.layers.Conv2D):
                    return sub.name
    raise ValueError("No Conv2D layer found in the model.")

def make_gradcam_heatmap(img_array, model, last_conv_layer_name=None, pred_index=None):
    if last_conv_layer_name is None:
        last_conv_layer_name = find_last_conv_layer_name(model)

    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.matmul(conv_outputs, pooled_grads[..., tf.newaxis])
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0)
    max_val = tf.reduce_max(heatmap)
    if max_val == 0:
        heatmap = heatmap
    else:
        heatmap /= max_val
    heatmap = heatmap.numpy()
    heatmap = cv2.resize(heatmap, (img_width, img_height))  # cv2 expects (width, height)
    return heatmap

def show_gradcam(model, generator, samples=3):
    last_conv = find_last_conv_layer_name(model)
    idxs = random.sample(range(len(generator.filenames)), min(samples, len(generator.filenames)))
    for idx in idxs:
        img_path = os.path.join(generator.directory, generator.filenames[idx])
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
        input_tensor = np.expand_dims(img_array, axis=0)

        heatmap = make_gradcam_heatmap(input_tensor, model, last_conv)

        heatmap_uint8 = np.uint8(255 * heatmap)
        heatmap_col = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)
        heatmap_col = cv2.cvtColor(heatmap_col, cv2.COLOR_BGR2RGB)

        orig_uint8 = np.uint8(img_array * 255.0)
        superimposed = cv2.addWeighted(orig_uint8, 0.6, heatmap_col, 0.4, 0)

        plt.figure(figsize=(10,4))
        plt.subplot(1,2,1); plt.imshow(orig_uint8); plt.title("Original"); plt.axis('off')
        plt.subplot(1,2,2); plt.imshow(superimposed); plt.title("Grad-CAM"); plt.axis('off')
        plt.show()

# run Grad-CAM on the best model found
val_generator.reset()
show_gradcam(best_model, val_generator, samples=5)